In [2]:
pip install pandas

  Using cached pandas-2.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.2-cp313-cp313-macosx_11_0_arm64.whl (10.7 MB)
Using cached numpy-2.3.2-cp313-cp313-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# combine and label data

import pandas as pd

df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

df_fake['label'] = 1  # fake
df_real['label'] = 0  # real

df = pd.concat([df_fake, df_real], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

In [ ]:
# look at data

df.head()
df['label'].value_counts()


label
1    23481
0    21417
Name: count, dtype: int64

In [6]:
!pip install nltk

  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.0 MB/s eta 0:00:00
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# clean text; focus on 'text'

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)  # remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['cleaned'] = df['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harmanloodu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
!pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 25.5 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# vectorize text!

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['cleaned'])
y = df['label']


In [ ]:
# split into testing and training sets (20% for testing)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
# create instance of model, then train it

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
# evaluate model using a confusion matrix

from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[4216   54]
 [  83 4627]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4270
           1       0.99      0.98      0.99      4710

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [ ]:
# compare with RandomForestClassifier and MultinomialNB

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# Create models
rf_model = RandomForestClassifier(random_state=42)
nb_model = MultinomialNB()

# Train models
rf_model.fit(X_train, y_train)
nb_model.fit(X_train, y_train)

# Predict on test data
rf_pred = rf_model.predict(X_test)
nb_pred = nb_model.predict(X_test)

# Evaluate Random Forest
print("Random Forest Classifier:")
print(confusion_matrix(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

# Evaluate MultinomialNB
print("Multinomial Naive Bayes:")
print(confusion_matrix(y_test, nb_pred))
print(classification_report(y_test, nb_pred))


Random Forest Classifier:
[[4255   15]
 [  13 4697]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4270
           1       1.00      1.00      1.00      4710

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

Multinomial Naive Bayes:
[[3945  325]
 [ 312 4398]]
              precision    recall  f1-score   support

           0       0.93      0.92      0.93      4270
           1       0.93      0.93      0.93      4710

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980



In [28]:
%pip install lime

1639.61s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 21.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.5 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283913 sha256=849ad9fa667c19a82059f589e27d16c4475a5ae9c15051f01e9dc29c1263f4b3
  Stored in directory: /Users/harmanloodu/Library/Caches/pip/wheels/7c/04/5c/157dc9106512a6c7a30653ec064490c94a49e0fc8f63d19ab9
Successfully built lime

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
import lime
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=['Real', 'Fake'])
idx = 0  # index of a test sample
exp = explainer.explain_instance(
    df['cleaned'].iloc[idx],
    model.predict_proba,
    num_features=10
)
exp.show_in_notebook()

ValueError: Expected 2D array, got 1D array instead:
array=['21st century wire says ben stein reputable professor pepperdine university also hollywood fame appearing tv shows films ferris bueller day made provocative statements judge jeanine pirro show recently discussing halt imposed president trump executive order travel stein referred judgement 9th circuit court washington state coup tat executive branch constitution stein went call judges seattle political puppets judiciary political pawns watch interview complete statements note stark contrast rhetoric leftist media pundits neglect note court ever blocked presidential orders immigration past discuss legal efficacy halt actual text executive order read trump news 21st century wire trump filessupport work subscribing becoming member 21wire tv'
 '21st   says ben stein   pepperdine university also hollywood fame  tv shows films ferris bueller day  provocative statements judge jeanine pirro show  discussing halt  president trump  order